In [ ]:
import os
import json
import re
import sys
import time

import torch
from torch import nn
from torch import optim
import matplotlib.pyplot as plt
import torchvision as tv
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np

from aegan import AEGAN

In [ ]:
BATCH_SIZE = 16
LATENT_DIM = 32
EPOCHS = 20000

In [ ]:
def save_images(GAN, vec, filename):
    images = GAN.generate_samples(vec)
    ims = tv.utils.make_grid(images[:36], normalize=True, nrow=6, )
    ims = ims.numpy().transpose((1, 2, 0))
    ims = np.array(ims * 255, dtype=np.uint8)
    image = Image.fromarray(ims)
    image.save(filename)


def gen_samples(dataset):
    dataloader = DataLoader(dataset,
                            batch_size=36,
                            shuffle=False,
                            num_workers=1,
                            drop_last=True
                            )
    X = iter(dataloader)
    test_ims, _ = next(X)
    # test_ims2, _ = next(X)
    # test_ims = torch.cat((test_ims1, test_ims2), 0)
    test_ims_show = tv.utils.make_grid(test_ims[:36], normalize=True, nrow=6, )
    test_ims_show = test_ims_show.numpy().transpose((1, 2, 0))
    test_ims_show = np.array(test_ims_show * 255, dtype=np.uint8)
    image = Image.fromarray(test_ims_show)
    return test_ims, image

In [ ]:
root = "small_data"  # os.path.join("data")
device = "cuda"  # torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = tv.transforms.Compose([
    # tv.transforms.RandomAffine(0, translate=(5/96, 5/96), fill=(255,255,255)),
    # tv.transforms.ColorJitter(hue=0.5),
    tv.transforms.RandomHorizontalFlip(p=0.5),
    tv.transforms.ToTensor(),
    tv.transforms.Normalize((0.5, 0.5, 0.5,), (0.5, 0.5, 0.5,))
])
dataset = ImageFolder(
    root=root,
    transform=transform
)
dataloader = DataLoader(dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        num_workers=1,
                        drop_last=True
                        )